In [1]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import Dataset

import evaluate
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

from sklearn.model_selection import train_test_split

SEED = 456
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [2]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'outputs')

model_name = "klue/roberta-large"

# Load Datasets

In [13]:
noised_path = os.path.join(DATA_DIR, 'kjs/noisy_df.csv')
noised_df = pd.read_csv(noised_path)
noised_df

NameError: name 'DATA_DIR' is not defined

In [12]:
cleaned_path = os.path.join(DATA_DIR, 'fill_mask/train_generated.csv')
cleaned_df = pd.read_csv(cleaned_path)
cleaned_df.head()

,ID,text,target
0,ynat-v1_train_00000,정권이 파리 미사 참석에 이용기간 제한을 두고 분열을 보임,4
1,ynat-v1_train_00001,찰 국 로한 회장이 맡은 주요 프로젝트,3
2,ynat-v1_train_00002,"""김정은, 자주통일 새 열쇠를 잡고 나서야 한다""",2
3,ynat-v1_train_00003,"""갤럭시 노트 7, 주말 동안 10만 대 개통…시장 불법 보조금 논란에 얼룩져""",5
4,ynat-v1_train_00004,"""대선을 앞두고 감시 강화, 단발적 비해""",6


In [13]:
augmented_path = os.path.join(DATA_DIR, 'fill_mask/train_filled.csv')
augmented_df = pd.read_csv(augmented_path)
augmented_df

,ID,text,target
0,ynat-v1_train_00000,"한국 축구 대표팀, 월드컵 예선전서 3-1 승리",4
1,ynat-v1_train_00001,"중국, 코로나19 백신 개발 성공 및 안전성 검증 완료",3
2,ynat-v1_train_00002,"정부, 올해 경제 성장률 3% 예상 발표",2
3,ynat-v1_train_00003,"갤노트 주말 100만 대 개통, 시장은 불법 보조금 얼룩",5
4,ynat-v1_train_00004,대선 앞두고 1억 5천만 원 추가로 감시 강화,6
...,...,...,...
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수 매겨 점만점에 50점,6
2796,ynat-v1_train_02796,삼성 갤럭시 S22 정식 출시 첫 주말 이통시장 침묵,2
2797,ynat-v1_train_02797,"텔레그램, 한 시간 동안 서비스 중단 및 정상화 과정",4
2798,ynat-v1_train_02798,"인터뷰 류현진, 친구에게 안타 맞는 것 싫어해 승부는 냉정한 것 같다",1


In [4]:
self_train_path = os.path.join(DATA_DIR, 'Self_train_2/self_train_step7.csv')
self_train_df = pd.read_csv(self_train_path)
self_train_df

,ID,text,target
0,ynat-v1_train_00000,정권이 파리 미사 참석에 이용기간 제한을 두고 분열을 보임,6
1,ynat-v1_train_00001,찰 국 로한 회장이 맡은 주요 프로젝트,3
2,ynat-v1_train_00002,"김정은, 자주통일 새 열쇠를 잡고 나서야 한다",2
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,4
4,ynat-v1_train_00004,"대선을 앞두고 감시 강화, 단발적 비해",6
...,...,...,...
8035,ynat-v1_train_08035,"다이아몬드 드래프트 판타지 랭킹: 추신수, 다르비쉬, 강정호",1
8036,ynat-v1_train_08036,평양에서의 반미 강의: 교실 이념의 개요,3
8037,ynat-v1_train_08037,경기도 아파트 관리비 사기 대규모 적발,3
8038,ynat-v1_train_08038,워커힐에서 새해 도시 겨울 축제를 즐겨보세요!,0


# Make Train dataset

In [4]:
## 문자열 앞뒤 따옴표 제거
import re
def remove_outer_quotes(input_string):
    if type(input_string) == float :
        return ' '
    # 문자열의 앞뒤 공백을 제거
    cleaned = input_string.strip()
    
    # 작은따옴표와 큰따옴표 확인
    while cleaned.startswith('"') or cleaned.startswith("'"):
        cleaned = cleaned[1:]
    while cleaned.endswith('"') or cleaned.endswith("'"):
        cleaned = cleaned[:-1]
    
    return cleaned

# 테스트
example_string = '""코로나19 대면 수업, 원격 수업 대비 뚜렷한 선호도 드러나"""'
result = remove_outer_quotes(example_string)
print(result)

코로나19 대면 수업, 원격 수업 대비 뚜렷한 선호도 드러나


In [5]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7).to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
dataset_train, dataset_valid = train_test_split(self_train_df, test_size=0.1, random_state=SEED)

class BERTDataset(Dataset):
    def __init__(self, data, tokenizer):
        input_texts = data['text']
        targets = data['target']
        self.inputs = []; self.labels = []
        for text, label in zip(input_texts, targets):
            tokenized_input = tokenizer(text, padding='max_length', truncation=True, return_tensors='pt')
            self.inputs.append(tokenized_input)
            self.labels.append(torch.tensor(label))

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx]['input_ids'].squeeze(0),
            'attention_mask': self.inputs[idx]['attention_mask'].squeeze(0),
            'labels': self.labels[idx].squeeze(0)
        }

    def __len__(self):
        return len(self.labels)

data_train = BERTDataset(dataset_train, tokenizer)
data_valid = BERTDataset(dataset_valid, tokenizer)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

f1 = evaluate.load('f1')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average='macro')


training_args = TrainingArguments(
    output_dir=OUTPUT_DIR+'/Self_train_step6',
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    report_to="wandb",
    logging_strategy='steps',
    eval_strategy='steps',
    save_strategy='steps',
    logging_steps=100,
    eval_steps=50,
    save_steps=50,
    save_total_limit=3,
    learning_rate= 2e-05,
    adam_beta1 = 0.9,
    adam_beta2 = 0.999,
    adam_epsilon=1e-08,
    weight_decay=0.01,
    lr_scheduler_type='linear',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    num_train_epochs=4,
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1',
    greater_is_better=True,
    seed=SEED
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data_train,
    eval_dataset=data_valid,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,F1
50,No log,0.664572,0.790511
100,1.814200,0.513936,0.841628
150,1.814200,0.398932,0.875294
200,1.105800,0.338204,0.879082
250,1.105800,0.315283,0.887239
300,0.631500,0.323385,0.898348
350,0.631500,0.314717,0.894006
400,0.471300,0.276474,0.904055
450,0.471300,0.312582,0.899789
500,0.342500,0.304441,0.904281


TrainOutput(global_step=904, training_loss=0.5351422011061052, metrics={'train_runtime': 3169.6278, 'train_samples_per_second': 9.132, 'train_steps_per_second': 0.285, 'total_flos': 2.6925815824920576e+16, 'train_loss': 0.5351422011061052, 'epoch': 3.9911699779249448})

In [15]:
## 데이터에서 불필요한 text row drop
denoising_df = denoising_df[denoising_df['text'].str.len() > 5]
denoising_df.to_csv(f'./data/Self_train/self_train_step2_cleaned.csv', index=False)

In [7]:
## 미리 학습했던 model을 이용해 re-labeling (Step 2)
model = AutoModelForSequenceClassification.from_pretrained(OUTPUT_DIR+"/Self_train_step6/checkpoint-900").to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.eval()

new_train = self_train_df.copy()
len(new_train)

8040

In [8]:

for idx, sample in tqdm(self_train_df.iterrows(), total=len(self_train_df), desc="Evaluating"):
    target_id = sample['ID']

    inputs = tokenizer(sample['text'], return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        logits = model(**inputs).logits
        pred = torch.argmax(torch.nn.Softmax(dim=1)(logits), dim=1).cpu().numpy()
        new_train.loc[new_train['ID'] == target_id, 'target'] = pred[0]


Evaluating: 100%|██████████| 8040/8040 [03:50<00:00, 34.95it/s]


In [9]:

new_train.to_csv(f'./data/Self_train_2/self_train_step8.csv', index=False)


# DATA Augmentation with BM25

In [7]:
from rank_bm25 import BM25Okapi
import numpy as np
corpus = self_train_df['text'].values.tolist()
corpus = [sentence for sentence in corpus if len(sentence) > 5]
len(corpus)

4750

In [8]:
# Hugging Face tokenizer 로드 (예: BERT tokenizer)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenized_corpus = [tokenizer.tokenize(doc) for doc in corpus]

# BM25 모델 생성
bm25 = BM25Okapi(tokenized_corpus)


In [11]:
# augmentation된 문장들
augmented_corpus = pd.read_csv("./data/cleaning_BT/e2k.csv")['text'].values
augmented_sentences = augmented_corpus.tolist()
augmented_sentences = [remove_outer_quotes(sentence) for sentence in augmented_sentences if (type(sentence)!=float and len(sentence) > 5)]
print(len(augmented_sentences))
augmented_sentences

4107


['파리 미사 출석에 시간 제한 부과한 정권, 의견 분열',
 '최옥국의 주요 프로젝트',
 '김정은, 자주적 통일 열쇠를 잡아야 한다',
 '갤럭시 노트 8 주말 판매량, 27만 대 돌파…불법 보조금으로 얼룩진 시장',
 '선거 전 강화된 감시, 간헐적 사례에 비해',
 '1명 중 6명의 미국인들이 배우자나 파트너의 빚을 떠맡고 있습니다',
 '비로 인해 KBO 구미-광주롯데 경기가 취소되었습니다',
 '아그라메즈 33점 득점, 우리카드가 KB 보험을 대승으로 물리치며 3위 자리 공고화!',
 '박 대통령, 포항 지진 피해 현장을 방문하며 얼마나 충격을 받았습니까',
 '듀얼-심 아이폰 출시 예정, 후반기에…저렴한 통신사들이 기대',
 '주간 주식시장 상승세, 매력적인 투자 기회 찾기 열기 고조',
 '1월 옵션 만기일에 NH투자, 매도 압력 경험하다',
 '황 총리, 모든 정부 기관에 걸쳐 포괄적인 비상 대비 확보',
 'AI, 대결에서 DeepMind의 AlphaZero를 뛰어넘다',
 '문재인 대통령의 당, 공공-민간 파트너십을 통한 철거 현장 재건 촉구',
 '과학기술정보통신부, 박민정 연구위원 과학기술진흥본부장 선출',
 '건설회사로부터 분쟁해결금 수령으로 기소된 기자, 집행유예 선고',
 '전환 임대 비용 상승…지난해 최고치 기록',
 '작가 정연, 그녀의 강아지와 함께 새로운 작품 선보여',
 '페이스북의 인터넷 드론, 아퀼라, 첫 번째 시험 비행에 성공적',
 '추신수, 시즌 타율 0.300 이상으로 마감… 최지만, 홈런 두 방으로 새로운 기록 수립',
 '수단 팬들, 축구 선수 호 장 한마에게 지지 보내다',
 '현대, 임원진 확대하며 경영진 역할 강화',
 '박항서의 아시안게임 포효: 일본을 놀라게 한 무적의 베트남',
 '서울, 오존 경보 재발…중앙, 북서부, 북동부 지역에 대한 경고 발령',
 '현장 점검을 위해 배치된 선박',
 '레컴의 분기별 이익이 두 배로 증가하다',
 '검증되지 않은 외국 학생, 결국 학위 취득',
 '축구협회,

In [12]:
# 유사도 계산 및 선택
threshold = 40
selected_sentences = []

for aug_sentence in tqdm(augmented_sentences):
    tokenized_query = tokenizer.tokenize(aug_sentence)
    scores = bm25.get_scores(tokenized_query)
    max_score = max(scores)

    if max_score < threshold:
        selected_sentences.append(aug_sentence)
        
# 선택된 문장들 추가
final_sentences = corpus + selected_sentences


100%|██████████| 4107/4107 [00:55<00:00, 74.58it/s]


In [13]:
len(final_sentences)

8040

In [14]:
## sentence list Dataframe으로 치환
data = {
    'ID': [f'ynat-v1_train_{i:05d}' for i in range(len(final_sentences))],
    'text': final_sentences,
    'target': [0] * len(final_sentences)
}

augmented_train_df = pd.DataFrame(data)
augmented_train_df

,ID,text,target
0,ynat-v1_train_00000,정권이 파리 미사 참석에 이용기간 제한을 두고 분열을 보임,0
1,ynat-v1_train_00001,찰 국 로한 회장이 맡은 주요 프로젝트,0
2,ynat-v1_train_00002,"김정은, 자주통일 새 열쇠를 잡고 나서야 한다",0
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,0
4,ynat-v1_train_00004,"대선을 앞두고 감시 강화, 단발적 비해",0
...,...,...,...
8035,ynat-v1_train_08035,"다이아몬드 드래프트 판타지 랭킹: 추신수, 다르비쉬, 강정호",0
8036,ynat-v1_train_08036,평양에서의 반미 강의: 교실 이념의 개요,0
8037,ynat-v1_train_08037,경기도 아파트 관리비 사기 대규모 적발,0
8038,ynat-v1_train_08038,워커힐에서 새해 도시 겨울 축제를 즐겨보세요!,0


In [15]:
model = AutoModelForSequenceClassification.from_pretrained("outputs/Self_train_step5/checkpoint-350").to(DEVICE)
model.eval()

augmented_train_relabelled = augmented_train_df.copy()

for idx, sample in tqdm(augmented_train_df.iterrows(), total=len(augmented_train_df), desc="Evaluating"):
    target_id = sample['ID']
    
    inputs = tokenizer(sample['text'], return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        logits = model(**inputs).logits
        pred = torch.argmax(torch.nn.Softmax(dim=1)(logits), dim=1).cpu().numpy()
        augmented_train_relabelled.loc[augmented_train_relabelled['ID'] == target_id, 'target'] = pred[0]

augmented_train_relabelled.to_csv(f'./data/Self_train_2/self_train_step6.csv', index=False)

Evaluating: 100%|██████████| 8040/8040 [03:48<00:00, 35.21it/s]


# Re-labelling 3

In [109]:
dataset_train, dataset_valid = train_test_split(augmented_train_relabelled, test_size=0.2, random_state=SEED)

class BERTDataset(Dataset):
    def __init__(self, data, tokenizer):
        input_texts = data['text']
        targets = data['target']
        self.inputs = []; self.labels = []
        for text, label in zip(input_texts, targets):
            tokenized_input = tokenizer(text, padding='max_length', truncation=True, return_tensors='pt')
            self.inputs.append(tokenized_input)
            self.labels.append(torch.tensor(label))

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx]['input_ids'].squeeze(0),
            'attention_mask': self.inputs[idx]['attention_mask'].squeeze(0),
            'labels': self.labels[idx].squeeze(0)
        }

    def __len__(self):
        return len(self.labels)

data_train = BERTDataset(dataset_train, tokenizer)
data_valid = BERTDataset(dataset_valid, tokenizer)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

f1 = evaluate.load('f1')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average='macro')

In [111]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR+'/Self_train_step3',
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    report_to="wandb",
    logging_strategy='steps',
    eval_strategy='steps',
    save_strategy='steps',
    logging_steps=100,
    eval_steps=50,
    save_steps=50,
    save_total_limit=2,
    learning_rate= 2e-05,
    adam_beta1 = 0.9,
    adam_beta2 = 0.999,
    adam_epsilon=1e-08,
    weight_decay=0.01,
    lr_scheduler_type='linear',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    num_train_epochs=10,
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1',
    greater_is_better=True,
    seed=SEED
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data_train,
    eval_dataset=data_valid,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,F1
50,No log,0.324325,0.888161
100,0.861300,0.344093,0.889226
150,0.861300,0.425726,0.875974
200,0.395400,0.400101,0.886577
250,0.395400,0.630774,0.863286
300,0.234400,0.507961,0.884126


KeyboardInterrupt: 

: 

In [5]:
model = AutoModelForSequenceClassification.from_pretrained("data/outputs/Self_train_step3/checkpoint-300").to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.eval()

st3_df = pd.read_csv(DATA_DIR + '/Self_train/self_train_step3.csv')
st3_df.head()


,ID,text,target
0,ynat-v1_train_00000,정권이 파리 미사 참석에 이용기간 제한을 두고 분열을 보임,6
1,ynat-v1_train_00001,찰 국 로한 회장이 맡은 주요 프로젝트,5
2,ynat-v1_train_00002,"김정은, 자주통일 새 열쇠를 잡고 나서야 한다",2
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,4
4,ynat-v1_train_00004,"대선을 앞두고 감시 강화, 단발적 비해",2


In [ ]:
st3_relabelled = st3_df.copy()

for idx, sample in tqdm(st3_df.iterrows(), total=len(st3_df), desc="Evaluating"):
    target_id = sample['ID']
    
    inputs = tokenizer(sample['text'], return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        logits = model(**inputs).logits
        pred = torch.argmax(torch.nn.Softmax(dim=1)(logits), dim=1).cpu().numpy()
        st3_relabelled.loc[st3_relabelled['ID'] == target_id, 'target'] = pred[0]

st3_relabelled.to_csv(f'./data/Self_train/self_train_step4.csv', index=False)

Evaluating: 100%|██████████| 4121/4121 [01:29<00:00, 45.98it/s]


: 